Use PySpark to load all of the data

In [1]:
import pyspark

sc = pyspark.SparkContext()

In [2]:
raw = sc.textFile("../states/*.csv")

splitified = raw.filter(lambda x: x != "state,total_votes,dem_votes,dem_pct,dem_ev,rep_votes,rep_pct,rep_ev,year").map(lambda x: x.replace(",,", ",0,").replace(",*,", ",0,").replace("*", "").replace("--", "0")).map(lambda x: x.split(","))
formatted = splitified.map(lambda x: ((x[0], int(x[8])), (int(x[1]), int(x[2]), float(x[3]), int(x[4]), int(x[5]), float(x[6]), int(x[7]))))

print(splitified.takeSample(False, 5))
print(formatted.takeSample(False, 5))

[['Florida', '276943', '206307', '74.5', '7', '69170', '25.0', '0', '1932'], ['Indiana', '3033121', '1242416', '40.96', '0', '1729519', '57.02', '11', '2020'], ['Mississippi', '1289865', '554662', '43.0', '0', '724597', '56.2', '6', '2008'], ['Oklahoma', '859350', '385581', '44.9', '0', '473769', '55.1', '8', '1956'], ['Kansas', '1327591', '544853', '41.04', '0', '758802', '57.16', '6', '2024']]
[(('Vermont', 1924), (102917, 16124, 15.7, 0, 80498, 78.2, 4)), (('Michigan', 1948), (2109609, 1003448, 47.6, 0, 1038595, 49.2, 19)), (('Vermont', 1972), (186947, 68174, 36.5, 0, 117149, 62.7, 3)), (('New York', 1964), (7166275, 4913102, 68.6, 43, 2243559, 31.3, 0)), (('New York', 2012), (7061925, 4471871, 63.4, 29, 2485432, 35.2, 0))]


In [3]:
#ratio of dem, rep, and other
ratios = formatted.mapValues(lambda x: (x[1] / x[0], x[4] / x[0], (x[0]-x[4]-x[1]) / x[0]))

print(ratios.takeSample(False, 5))

alabama = ratios.filter(lambda x: x[0][0] == "Alabama").map(lambda x: (x[0][1], x[1]))

print(alabama.takeSample(False, 5))

[(('Idaho', 2004), (0.3026132639983156, 0.6838283089396388, 0.013558427062045594)), (('Mississippi', 2020), (0.41057606455978607, 0.5760295457538255, 0.013394389686388447)), (('Idaho', 2012), (0.3262233355920978, 0.6452978349589283, 0.028478829448973897)), (('Connecticut', 1996), (0.5283158147196567, 0.34690804487101234, 0.12477614040933094)), (('Indiana', 2004), (0.39262974665336575, 0.5994476503665718, 0.007922602980062416))]
[(1944, (0.8127627756462902, 0.18198681882627898, 0.005250405527430815)), (2000, (0.4156650294789806, 0.5648375535326765, 0.019497416988342838)), (2024, (0.3410072005968858, 0.6457209205815222, 0.013271878821592078)), (2012, (0.3835903309875247, 0.6054582232982282, 0.010951445714247148)), (1960, (0.568284449120961, 0.41734578455872684, 0.014369766320312158))]


In [5]:
state_text_path = "../states.txt"
states_save_path = "../by_state"

f = open(state_text_path)
states = f.readlines()
for state in states:
    print(f"State: {state}")
    state = state.strip()
    staterdd = formatted.filter(lambda x: x[0][0] == state).map(lambda x: (x[0][1], x[1])).coalesce(1)
    staterdd.saveAsTextFile(states_save_path + "/" + state)

State: Alabama

State: Alaska

State: Arizona

State: Arkansas

State: California

State: Colorado

State: Connecticut

State: Delaware

State: District of Columbia

State: Florida

State: Georgia

State: Hawaii

State: Idaho

State: Illinois

State: Indiana

State: Iowa

State: Kansas

State: Kentucky

State: Louisiana

State: Maine

State: Maryland

State: Massachusetts

State: Michigan

State: Minnesota

State: Mississippi

State: Missouri

State: Montana

State: Nebraska

State: Nevada

State: New Hampshire

State: New Jersey

State: New Mexico

State: New York

State: North Carolina

State: North Dakota

State: Ohio

State: Oklahoma

State: Oregon

State: Pennsylvania

State: Rhode Island

State: South Carolina

State: South Dakota

State: Tennessee

State: Texas

State: Utah

State: Vermont

State: Virginia

State: Washington

State: West Virginia

State: Wisconsin

State: Wyoming
